<H1> Example on using the datasets for quantum characterization and control </H1>

In [1]:
# preample
import zipfile
import pickle
import os
import numpy as np
from typing import Tuple

In [2]:
datalist = [
'G_1q_X',
# 'G_1q_XY',
# 'G_1q_XY_XZ_N1N5',
# 'G_1q_XY_XZ_N1N6',
# 'G_1q_XY_XZ_N3N6',
# 'G_1q_X_Z_N1',
# 'G_1q_X_Z_N2',
# 'G_1q_X_Z_N3',
# 'G_1q_X_Z_N4',
# 'G_2q_IX-XI_IZ-ZI_N1-N6',
# 'G_2q_IX-XI-XX',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_1q_X',
# 'S_1q_XY',
# 'S_1q_XY_XZ_N1N5',
# 'S_1q_XY_XZ_N1N6',
# 'S_1q_XY_XZ_N3N6',
# 'S_1q_X_Z_N1',
# 'S_1q_X_Z_N2',
# 'S_1q_X_Z_N3',
# 'S_1q_X_Z_N4',
# 'S_2q_IX-XI_IZ-ZI_N1-N6',
# 'S_2q_IX-XI-XX',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N6',
]

In [3]:
'''Create two strings, one for each of the undistorted and distorted datasets.'''
data1 = datalist[0]
data2 = data1 + '_D'
print(data1)
print(data2)

G_1q_X
G_1q_X_D


In [4]:
'''Set the working directory to the location of the datasets, example below.'''
print(os.getcwd())
os.chdir(f'./QuantumDS/{data1}')

/mnt/c/Users/ChrisWiseXPSLocal/GitHub/QDataSet


## Using the $V_O$ operators in a calculation

In [5]:
def load_Vo_dataset(
    dataset_name: str, num_examples: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Load the dataset of pulses and Vo operators.

    Args:
        dataset_name: Name of the dataset.
        num_examples: Number of examples to load.

    Returns:
        pulses: Array of pulses.
        Vx: Array of X Vo operators.
        Vy: Array of Y Vo operators.
        Vz: Array of Z Vo operators.
    """
    pulses = np.zeros((num_examples, 1024, 1), dtype=np.complex64)
    Vx = np.zeros((num_examples, 2, 2), dtype=np.complex64)
    Vy = np.zeros((num_examples, 2, 2), dtype=np.complex64)
    Vz = np.zeros((num_examples, 2, 2), dtype=np.complex64)

    with zipfile.ZipFile(f"{dataset_name}.zip", mode="r") as fzip:
        for index, fname in enumerate(fzip.namelist()[:num_examples]):
            with fzip.open(fname, "r") as f:
                data = pickle.load(f)
                pulses[index, :, :] = data["pulses"][0:1, :, 0, :]
                Vx[index], Vy[index], Vz[index] = data["Vo_operator"]
    return pulses, Vx, Vy, Vz

In [6]:
# define the dataset parameters
dataset_name = data2  # "G_1q_XY_XZ_N1N5_D" # dataset name
num_ex = 3  # number of examples

In [7]:
pulses, Vx, Vy, Vz = load_Vo_dataset(dataset_name, num_ex)

In [8]:
print(pulses.shape, Vx.shape, Vy.shape, Vz.shape)
print(pulses)
print(Vx)
print(Vy)
print(Vz)

(3, 1024, 1) (3, 2, 2) (3, 2, 2) (3, 2, 2)
[[[ 1.3720762e-07+0.j]
  [ 1.9746030e-07+0.j]
  [ 2.8319752e-07+0.j]
  ...
  [ 1.7342549e-26+0.j]
  [ 8.9434386e-27+0.j]
  [ 4.5962476e-27+0.j]]

 [[ 1.7338933e-09+0.j]
  [ 2.6096181e-09+0.j]
  [ 3.9141770e-09+0.j]
  ...
  [-1.5898091e-33+0.j]
  [-7.5606038e-34+0.j]
  [-3.5832486e-34+0.j]]

 [[-3.5911532e-10+0.j]
  [-5.4713234e-10+0.j]
  [-8.3072987e-10+0.j]
  ...
  [ 1.8026266e-35+0.j]
  [ 8.4224185e-36+0.j]
  [ 3.9216921e-36+0.j]]]
[[[ 9.9987763e-01+4.5830006e-13j -2.5628565e-06+0.0000000e+00j]
  [-2.5628565e-06+0.0000000e+00j  9.9987763e-01-4.5830006e-13j]]

 [[ 9.9987787e-01-2.1249669e-12j  1.7284700e-06+0.0000000e+00j]
  [ 1.7284691e-06+0.0000000e+00j  9.9987787e-01+2.1249669e-12j]]

 [[ 9.9987644e-01-6.4659394e-13j -1.5496510e-06+0.0000000e+00j]
  [-1.5496510e-06+0.0000000e+00j  9.9987644e-01+6.4659394e-13j]]]
[[[0.99987763-4.5830006e-13j 0.        +3.5760922e-07j]
  [0.        -3.5760922e-07j 0.99987763+4.5830006e-13j]]

 [[0.99987787+2

In [11]:
import tensorflow.keras as K
embed_dim = 1024

input_layer = K.Input(shape=(None, axnum))
embedding_layer = K.layers.Embedding(input_dim=axnum, output_dim=embed_dim)(
    input_layer
)
transformer_layer = K.layers.Transformer(
    num_heads=num_heads, feed_forward_dim=feed_forward_dim, activation="relu"
)(embedding_layer)
output_layer = K.layers.Reshape((axnum3, axnum3))(transformer_layer)
ml_model = K.Model(input_layer, output_layer)
ml_model.compile(optimizer=K.optimizers.Adam(), loss="mse")
ml_model.fit(pulses, [Vx, Vy, Vz], epochs=10, validation_split=0.1)

ImportError: cannot import name 'layers' from partially initialized module 'keras' (most likely due to a circular import) (/mnt/c/Users/ChrisWiseXPSLocal/GitHub/QDataSet/venv/lib/python3.10/site-packages/keras/__init__.py)

## Model the effect of control distortions

In [ ]:
# define the dataset parameters
dataset_name    = data2 #"G_1q_XY_XZ_N1N5_D" # dataset name
num_training_ex = 7                   # number of training examples
num_testing_ex  = 3                   # number of testing examples

In [ ]:
# load the dataset
training_input, training_target, testing_input, testing_target = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [ ]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)

(7, 1024, 1) (7, 18) (3, 1024, 1) (3, 18)


In [ ]:
'''Set shape parameters based on shape of inputs and outputs.'''
axnum = training_input.shape[2]
lstmout = training_target.shape[1]


In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(None, axnum))

output_layer  = K.layers.LSTM(lstmout, return_sequences=False)(input_layer)

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.3332 - val_loss: 0.3333
Epoch 2/10
1/1 [==============================] - 1s 505ms/step - loss: 0.3328 - val_loss: 0.3334
Epoch 3/10
1/1 [==============================] - 1s 518ms/step - loss: 0.3324 - val_loss: 0.3335
Epoch 4/10
1/1 [==============================] - 1s 528ms/step - loss: 0.3320 - val_loss: 0.3336
Epoch 5/10
1/1 [==============================] - 1s 514ms/step - loss: 0.3316 - val_loss: 0.3337
Epoch 6/10
1/1 [==============================] - 1s 503ms/step - loss: 0.3311 - val_loss: 0.3338
Epoch 7/10
1/1 [==============================] - 1s 592ms/step - loss: 0.3306 - val_loss: 0.3340
Epoch 8/10
1/1 [==============================] - 1s 526ms/step - loss: 0.3302 - val_loss: 0.3341
Epoch 9/10
1/1 [==============================] - 0s 493ms/step - loss: 0.3297 - val_loss: 0.3343
Epoch 10/10
1/1 [==============================] - 1s 540ms/step - loss: 0.3291 - val_loss: 0.3345


## Learning a controller for a quantum system

In [ ]:
# define the dataset parameters
dataset_name    = data1 #"G_1q_XY_XZ_N1N5"   # dataset name
num_training_ex = 7                   # number of training examples
num_testing_ex  = 3                   # number of testing examples

In [ ]:
# load the dataset [inputs and targets are echanged for quantum control problem, this is the inverse of the 
# characterization problem]
training_target, training_input, testing_target, testing_input  = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [ ]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)



(7, 18) (7, 1024, 1) (3, 18) (3, 1024, 1)


In [ ]:
'''Set shape parameters based on shape of inputs and outputs.'''
axnum4 = training_input.shape[1]
axnum5 = training_target.shape[1]
axnum6 = training_target.shape[2]
axnum7 = axnum5 * axnum6

In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(axnum4))

output_layer  = K.layers.Reshape((axnum5,axnum6))( K.layers.Dense(axnum7)(input_layer) )

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 1s 700ms/step - loss: 439.0493 - val_loss: 523.0949
Epoch 2/10
1/1 [==============================] - 0s 38ms/step - loss: 438.9573 - val_loss: 523.0969
Epoch 3/10
1/1 [==============================] - 0s 34ms/step - loss: 438.8654 - val_loss: 523.0988
Epoch 4/10
1/1 [==============================] - 0s 34ms/step - loss: 438.7735 - val_loss: 523.1008
Epoch 5/10
1/1 [==============================] - 0s 32ms/step - loss: 438.6816 - val_loss: 523.1027
Epoch 6/10
1/1 [==============================] - 0s 35ms/step - loss: 438.5898 - val_loss: 523.1047
Epoch 7/10
1/1 [==============================] - 0s 46ms/step - loss: 438.4980 - val_loss: 523.1068
Epoch 8/10
1/1 [==============================] - 0s 41ms/step - loss: 438.4063 - val_loss: 523.1088
Epoch 9/10
1/1 [==============================] - 0s 46ms/step - loss: 438.3147 - val_loss: 523.1108
Epoch 10/10
1/1 [==============================] - 0s 44ms/step - loss: 438.2230 - val_los